In [ ]:
# Parameters.  Set defaults here.
import datetime

dayobs_date = datetime.date.fromisoformat("2025-09-21")
instrument = "LSSTCam"

In [ ]:
date = dayobs_date.isoformat()
day_obs = date.replace("-", "")  # day_obs is a string in the EFD

# Alert Production Data Quality Report for {{params.instrument}} on {{ date }}

In [ ]:
match instrument:
    case "LSSTCam":
        sal_index = 1
        n_detector = 189
    case "LSSTComCam":
        sal_index = 1
        n_detector = 9
    case "LATISS":
        sal_index = 2
        n_detector = 1
    case "LSSTComCamSim":
        sal_index = 3
        n_detector = 9
    case _:
        raise ValueError(f"Unknown instrument {instrument}")

In [ ]:
# https://rtn-045.lsst.io/#colorblind-friendly-plots
plot_filter_colors = {
    "u": "#1600ea",
    "g": "#31de1f",
    "r": "#b52626",
    "i": "#370201",
    "z": "#ba52ff",
    "y": "#61a2b3",
}
plot_symbols = {"u": "o", "g": "^", "r": "v", "i": "s", "z": "*", "y": "p"}

1.3.14 Level 1 Data Quality Report Definition

ID: DMS-REQ-0097 (Priority: 1a)

Specification: The DMS shall produce a Level 1 Data Quality Report that contains indicators
of data quality that result from running the DMS pipelines, including at least: Photometric
zero point vs. time for each utilized filter; Sky brightness vs. time for each utilized filter; seeing
vs. time for each utilized filter; PSF parameters vs. time for each utilized filter; detection
efficiency for point sources vs. mag for each utilized filter.

Discussion: The seeing report is intended as a broad-brush measure of image quality. The
PSF parameters provide more detail, as they include asymmetries and field location
dependence

The plots below are generated from metrics computed in single-frame processing in Prompt Processing.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from lsst_efd_client import EfdClient
from astropy.time import Time
import astropy.units as u

In [ ]:
t = Time(date)
search_start_time = t.isot + "Z"
# TODO: look at time zones and understand if a narrower range can capture all of one dayobs
search_end_time = (t + 2 * u.day).isot + "Z"

In [ ]:
# As of 9 Oct 24, `lsst.prompt` is only available in usdfdev while we await authentication.
client = EfdClient("usdfdev_efd", db_name="lsst.prompt")


# waiting on input in #sasquatch_support
# client = EfdClient("usdf_efd", db_name="lsst.prompt")

In [ ]:
# await client.get_topics()

In [ ]:
# await client.get_fields("lsst.prompt.prod.initialPviSummaryMetrics")

In [ ]:
# https://docs.influxdata.com/influxdb/v1/query_language/explore-data/#the-where-clause
#
# > Single quote string field values in the WHERE clause. Queries with unquoted string field values or double quoted string field values will not return any data and, in most cases, will not return an error.

In [ ]:
# Testing to find instruments
# query = f"""SELECT "instrument" FROM "lsst.prompt.prod.initialPviSummaryMetrics" """

# print(query)
# initialPviSummaryMetrics = await client.influx_client.query(query)
# set(initialPviSummaryMetrics['instrument'])

In [ ]:
# Testing to find good dates
# query = f"""SELECT * FROM "lsst.prompt.prod.initialPviSummaryMetrics" WHERE "instrument" = '{instrument}' """
# query = f"""SELECT "instrument" FROM "lsst.prompt.prod.initialPviSummaryMetrics" """


# print(query)
# initialPviSummaryMetrics = await client.influx_client.query(query)
# set(initialPviSummaryMetrics['instrument'])

In [ ]:
query = f"""SELECT * FROM "lsst.prompt.prod.initialPviSummaryMetrics" WHERE 
"time" > '{search_start_time}' AND "time" < '{search_end_time}'  AND
"day_obs" = '{day_obs}' AND "instrument" = '{instrument}' """
# print(query)
# initialPviSummaryMetrics = await client.influx_client.query(query)

chunks = await client.influx_client.query(query, chunked=True, chunk_size=10000)
initialPviSummaryMetrics = pd.concat([pd.DataFrame(chunk) async for chunk in chunks])

In [ ]:
# initialPviSummaryMetrics

In [ ]:
if len(initialPviSummaryMetrics) == 0:
    raise SystemExit(f"No data for {date}.")

In [ ]:
# query = f"""SELECT zeroPoint FROM "lsst.prompt.prod.initialPviSummaryMetrics" WHERE
#        "time" > '{search_start_time}' AND "time" < '{search_end_time}'  AND "day_obs" = '{day_obs}' AND "instrument" = '{instrument}' """
# print(query)
# initialPviSummaryMetrics = await client.influx_client.query(query)

In [ ]:
# query = f"""EXPLAIN ANALYZE SELECT * FROM "lsst.prompt.prod.initialPviSummaryMetrics" WHERE "day_obs" = '{day_obs}' AND "instrument" = '{instrument}' """
# print(query)
# initialPviSummaryMetrics = await client.influx_client.query(query)

In [ ]:
# initialPviSummaryMetrics

In [ ]:
def plot_metric_vs_time_by_filter(df, column, unit=None):
    fig = plt.figure()
    for filt in plot_filter_colors.keys():
        wf = df["band"] == filt

        if np.sum(wf):
            plt.plot(
                df.loc[wf].index,
                df.loc[wf, column],
                plot_symbols[filt],
                color=plot_filter_colors[filt],
                label=filt,
            )

    plt.legend()
    plt.xlabel("Time")
    ylabel = f"{column} [{unit}]" if unit else column
    plt.ylabel(ylabel)

In [ ]:
# units are taken from https://github.com/lsst/analysis_tools/blob/main/python/lsst/analysis/tools/atools/calexpMetrics.py

## Photometric Zeropoint vs. Time, by Filter

In [ ]:
plot_metric_vs_time_by_filter(initialPviSummaryMetrics, "zeroPoint", unit="mag")

## Sky Brightness vs. Time, by Filter

In [ ]:
plot_metric_vs_time_by_filter(initialPviSummaryMetrics, "skyBg", unit="electrons")

In [ ]:
plot_metric_vs_time_by_filter(initialPviSummaryMetrics, "skyNoise", unit="electrons")

## Seeing vs. Time, by Filter

In [ ]:
plot_metric_vs_time_by_filter(initialPviSummaryMetrics, "psfSigma", unit="pixels")

## PSF Parameters vs. Time, by Filter

In [ ]:
plot_metric_vs_time_by_filter(initialPviSummaryMetrics, "psfIxx", unit="pixel$^2$")

In [ ]:
plot_metric_vs_time_by_filter(initialPviSummaryMetrics, "psfIyy", unit="pixel$^2$")

In [ ]:
plot_metric_vs_time_by_filter(initialPviSummaryMetrics, "psfIxy", unit="pixel$^2$")

In [ ]:
plot_metric_vs_time_by_filter(initialPviSummaryMetrics, "psfArea", unit="pixel$^2$")

## Detection efficiency for point sources vs. mag, by filter.

This calculation requires a source injection campaign for full fidelity, but the limiting magnitudes are a useful proxy.

In [ ]:
if "magLim" in initialPviSummaryMetrics:
    plot_metric_vs_time_by_filter(initialPviSummaryMetrics, "magLim", unit="mag")
else:
    # missing for dates before DM-52885
    print(f"No magLim values stored for {date}.")